In [115]:
import random
from bisect import bisect_left


#Generate N elements for each layer and select M out of N
class Element: #define a element
 def __init__(self, eID, layer, monitored):
    self.id = eID
    self.layer = layer
    self.monitored = monitored

 def infoOut(self):
    print("Element info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", monitored = " 
          + str(self.monitored) 
         ) 

class EvtQueue:
 def __init__(self):
    self.evtID = 0 #initial number of events
    self.queue = [] #empty queue
                
 def getNextEvt(self): #obtain the head of line fault in the evt queue
    if (len(self.queue) > 0):
        currFault = self.queue[0]
        self.queue.remove(currFault) #delete the currFault 
        return currFault 
    else:
        return None
    
 def hasNextEvt(self): #check if the queue is not empty
    if (len(self.queue) > 0):
        return True
    else:
        return False
        
 def getEvtNum(self):
    return self.evtID
    
 def infoOut(self):
    for f in self.queue:
        f.infoOut()
    
 def insertEvt(self, newFault):
    def insert(seq, keys, item, keyfunc=lambda v: v):
        """Insert an item into a sorted list using a separate corresponding
       sorted keys list and a keyfunc() to extract the key from each item.
       Based on insert() method in SortedCollection recipe:
       http://code.activestate.com/recipes/577197-sortedcollection/
        """
        k = keyfunc(item)  # Get key.
        i = bisect_left(keys, k)  # Determine where to insert item.
        keys.insert(i, k)  # Insert key of item to keys list.
        seq.insert(i, item)  # Insert the item itself in the corresponding place.
    keys = [fault.evtTime for fault in self.queue]
    insert(self.queue, keys, newFault, keyfunc=lambda x: x.evtTime) #insert on the order of evtTime to the queue
    self.evtID = self.evtID + 1
    
#Generate a seqence of faults sorted in an increasing order of evtTime
class Fault: #define a fault
 def __init__(self, fID, initlayer, layer, initTime, evtTime, assocElement, impactVector):
    self.id = fID
    self.initLayer = initlayer
    self.layer = layer
    self.initTime = initTime #the inital time when the fault is generated for calculating delay
    self.evtTime = evtTime #the time when the fault is re-inserted in the event queue
    self.assocElement = assocElement #within N elements
    self.impactVector = impactVector #impact vector of this fault
    
 def infoOut(self):
    print("Fault info: id = "
          + str(self.id)
          + ", initLayer = " 
          + str(self.initLayer) 
          + ", layer = " 
          + str(self.layer) 
          + ", initTime = " 
          + str(self.initTime) 
          + ", evtTime = " 
          + str(self.evtTime) 
          + ", assocElementID = " 
          + str(self.assocElement.id)
          + ", monitored = " 
          + str(self.assocElement.monitored)
          + ", impactVector = " 
          + str(self.impactVector)          
         )  
    
#This class manages all elements init, update in the system
class Elements:
    def __init__(self, N, M_N):
        self.M_N = M_N
        self.M = []
        self.N = N
        for i in range(len(N)):
            self.M.append(self.N[i]*self.M_N)
    
        self.elements = []
        elementID = 0
        for i in range(len(N)): #for each layer i
            j = 0
            elements_layer = []
            while j < self.N[i]: #generate Ni elements
                elementID = elementID + 1
                elements_layer.append(Element(elementID, i, False))
                j = j + 1
            count = 0
            assert(self.M[i] <= self.N[i]), "Total elements Ni is too small to reach Mi!"
            while count < self.M[i]: #select Mi elements out of Ni 
                randomElement = random.randrange(N[i])
                #print(randomElement)
                if elements_layer[randomElement].monitored == False:
                    elements_layer[randomElement].monitored = True
                    count = count + 1
            self.elements.append(elements_layer)
    
    #Update elements based on a given M_N, randomly select M out of N, but no change of the array 
    def update_elements(self, M_N):
        self.M_N = M_N
        #print(self.M)
        #print (self.M_N)
        #print(self.N)
        for i in range(len(self.M)):
            self.M[i] = self.N[i]*self.M_N
        #print(self.M)
        
        for i in range (I): #for each layer i
            j = 0
            elements_layer = self.elements[i]
            while j < self.N[i]: #generate Ni elements
                #elementID = elementID + 1
                elements_layer[j].monitored = False #reset all to False
                j = j + 1
            
            count = 0
            assert(self.M[i] <= self.N[i]), "Total elements Ni is too small to reach Mi!"
            while count < self.M[i]: #select Mi elements out of Ni 
                randomElement = random.randrange(self.N[i])
                #print(randomElement)
                if elements_layer[randomElement].monitored == False:
                    elements_layer[randomElement].monitored = True
                    count = count + 1
        
    def infoOut(self):
        for i in range (I): #for each layer i
            j = 0
            elements_layer = self.elements[i]
            strOut = ""
            while j < self.N[i]: #generate Ni elements
                #elementID = elementID + 1
                strOut = strOut + str(elements_layer[j].monitored) + " " 
                j = j + 1
            strOut = strOut + "\n"
            print(strOut)
        
def One_Experiment(_option, _failureRatio, _M_N, _accuracyRCA, _accuracyCorrelation):
    
 def generate_impactVector(numLayer): #generate an impact vector with a random impact value at element j at layer i, where element j is randomly selected among the Ni number of elements of layer i
    vectors = []
    #if (i in range (0, Ni): #assign all zeros to the ImpactVector
    #    vector[i] = 0
    for i in range(numLayer):
        vector = []        
        vector.append(i) #vector[0] = i, the layer i
        vector.append(random.randint(0, N[i]-1)) #vector[1] = j, randomly select the element j
        vector.append(random.random()) #vector[2] = the impact value
        vectors.append(vector)
    return vectors
                   
 def generate_newFault(evtTime, elements): #generate a new fault at evtTime among a set of elements
    layer = random.randint(0, I-1) #randomly select a layer, 0 ~ I-1
    elements_layer = elements.elements[layer] #find corresponding elements_layer 
    elementIndex = random.randint(0, len(elements_layer)-1) #randomly (uniformaly) assign fault to Ni elements a..t layer i
    newFault = Fault(faultsTotal, layer, layer, evtTime, evtTime, elements_layer[elementIndex], generate_impactVector(I))  
    return newFault

 def generate_impactedFault(vector, elements): #generate a impacted fault at evtTime among a set of elements based on an impact vector at layer i
    DELTA = random.uniform(1, DELTA_RANGE)
    nextEvtTime = currFault.evtTime + DELTA
    #for the new impacted fault, the impact vector is None
    newFault = Fault(faultsTotal, currFault.initLayer, vector[0], currFault.initTime, nextEvtTime, 
                     elements.elements[vector[0]][vector[1]], None)  
    return newFault

 """ 
 Initialization
 """
 #Input loop variables
 option = _option
 failureRatio = _failureRatio
 M_N = _M_N    
 accuracyRCA = _accuracyRCA
 accuracyCorrelation = _accuracyCorrelation

 #System architecture
 I = 5 # number of layers 0 ~ (I-1)
 N = [10, 10, 10, 10, 10] #number of elements for layer i
    
 #Simulation configuration
 LASTTIME = 1000000; #simulation running time
 SEED = 1
 random.seed(SEED)
    
 #Algorithm configuration
 THRESHOLD_IMPACTVECTOR = 0.5
 DELTA_RANGE = 10 #the time range (0, DELTA_RANGE) for a fault to be generated by impact factor 
 D_0 = 10 # the largest delay
 D_1 = 8 # a large delay
 D_2 = 5
 D_3 = 3 
 #accuracyFaultDetection = 0.9
 THRESHOLD_UP = 0.9 #M_N*0.8 #Threshold # C_R/faultsTotal = M/N * accuracyFaultDetection * accuracyRCA 
 THRESHOLD_DOWN = 0.6 #M_N*0.3 #double check here
 M_N_Change_Rate = 0.3
 THRESHOLD_FAULTS = 0.1 * LASTTIME #check and update M_N frequence relative to the overall system time

 #Counter for monitored/detected/diagnosed faults: 
 C_M = 0 #monitored 
 C_D = 0 #detected
 C_R = 0 #diagnosed
 faultsTotal = 0 #faults != events in queue, one fault may trigger multiple events at different layers for option 4
 delayTotal = 0
 impactedFaultsTotal = 0
 timesThresholdFaults = 1
    
 #Output
 M_N_history = [] #the change of M_N over time for monitoring policy
 timePoint = []
 timePoint.append(0)
 timePoint.append(0)
 timePoint.append(M_N)
 timePoint.append(0)
 M_N_history.append(timePoint)

 #Start simuation
 #initialize elements monitored and not-monitored at each layer
 elements = Elements(N, M_N)
 #elements.infoOut()
 queue = EvtQueue()

 # insert an initial fault with evtTime = 0 in the queue
 evtTime = 0
 newFault = generate_newFault(evtTime, elements)
 queue.insertEvt(newFault)
 faultsTotal = faultsTotal + 1
 #queue.infoOut()


 #start to process a Head of Line fault in the event queue
 while (queue.hasNextEvt()):
    currFault = queue.getNextEvt()
    assert(currFault != None), "Empty queue!"
    #currFault.infoOut()
   
    ### 
    #Fault generation
    ###
    #New fault generation  
    ### 
    #insert the next new fault based on the info of the currFault
    #this new fault generaton is to follow the poission process of new fault generation
    #it is not related to the fault generated due to impact vector
    #only generate a new fault for the current fault with its initTime == evtTime
    if currFault.evtTime == currFault.initTime:
        nextEvtTime = currFault.evtTime + random.expovariate(failureRatio)
        if nextEvtTime <= LASTTIME:
            elementIndex = random.randint(0, len(elements.elements[currFault.layer])-1) #generate a new fault with the same layer as the current fault
            newFault = generate_newFault(nextEvtTime, elements)
            queue.insertEvt(newFault)
            faultsTotal = faultsTotal + 1
            #newFault.infoOut()
    elif currFault.evtTime < currFault.initTime:
        print ("currFault.evtTime < currFault.initTime: Wrong evtTime!")
        
    ### 
    #Monitoring and process the currFault
    ###        
    if option == 1:
        #Calculate counters
        if currFault.assocElement.monitored == True:
            delayTotal = delayTotal + D_1
            C_M = C_M + 1
        else:
            delayTotal = delayTotal + D_0
        #currFault.infoOut()
    elif option == 2 or option == 3 or option == 4:
        #Calculate counters
        if currFault.assocElement.monitored == True:
            accuracy_rca = random.random()
            if (accuracy_rca < accuracyRCA):
                delayTotal  = delayTotal + D_2
                C_R = C_R + 1
            else:   
                delayTotal = delayTotal + D_1
                C_M = C_M + 1
        else:
            delayTotal = delayTotal + D_0
        #currFault.infoOut()
    
    ### 
    #Impacted fault generation
    ### 
    #generate impacted fault from the curFault based on the impact vector
    #Impacted fault does not have impact vector further generated since we only conside one round of impact across layer 
    if option == 1 or option == 2 or option == 3:
        if (currFault.impactVector != None): #not for the impacted faults with None impact vector
            for i in range(len(currFault.impactVector)):
                vector = currFault.impactVector[i]
                if (vector[2] >= THRESHOLD_IMPACTVECTOR):
                    newFault = generate_impactedFault(vector, elements)
                    queue.insertEvt(newFault)
                    faultsTotal = faultsTotal + 1
                    impactedFaultsTotal = impactedFaultsTotal + 1
    
    ###         
    #Impact fault analysis and generation 
    ### 
    if option == 4:
        if (currFault.impactVector != None): 
            for i in range(len(currFault.impactVector)):
                vector = currFault.impactVector[i]
                if (vector[2] >= THRESHOLD_IMPACTVECTOR):
                    insert = True
                    newFault = generate_impactedFault(vector, elements)
                    accuracy_correlation = random.random() #randomly generate accuracy of correlation
                    accuracy_rca = random.random() #randomly generate accuracy of RCA
                    if (accuracy_correlation < accuracyCorrelation):
                        if (accuracy_rca < accuracyRCA):
                            insert = False
                            delayTotal = delayTotal + D_2
                            C_R = C_R + 1
                    if insert == True: #insert impacted fault only if the corellation analysis fails
                        #faultsTotal = faultsTotal + 1  #double check
                        queue.insertEvt(newFault)                        
                        impactedFaultsTotal = impactedFaultsTotal + 1 #double check
                    faultsTotal = faultsTotal + 1 #double check
                    
    #For option = 3 with Fixed M/N
    #THRESHOLD_UP = 1
    #THRESHOLD_DOWN = 0    
    
    #update monitoring policy
    if option == 3 or option == 4:
        if currFault.evtTime > timesThresholdFaults*THRESHOLD_FAULTS:
            #print(str(faultsTotal) + ", " + str(timesThresholdFaults*THRESHOLD_FAULTS))
            #print("C_R/faultsTotal = " + str(C_R/faultsTotal) + 
            #      ", THRESHOLD_UP = " + str(THRESHOLD_UP) + 
            #      ", THRESHOLD_DOWN = " + str(THRESHOLD_DOWN))
            if C_R/faultsTotal > THRESHOLD_UP: 
                #too high, can decrease M_N and accuracy
                #update accuracy for detection and rca
                #decrease M_N
                #infoOut = "Decrease M/N from " + str(M_N) + " to "
                newM_N = max(0, M_N * (1 - M_N_Change_Rate))
                M_N = newM_N
                elements.update_elements(newM_N)
                
                #output
                #infoOut = infoOut + str(newM_N)
                #print(infoOut)
                #elements.infoOut()
                timePoint = []
                timePoint.append(timesThresholdFaults)
                timePoint.append(C_R/faultsTotal)
                timePoint.append(M_N)
                timePoint.append(delayTotal)
                M_N_history.append(timePoint)
                
            elif C_R/faultsTotal < THRESHOLD_DOWN: 
                #too low, need to increase M_N and accuracy
                #update accuracy for detection and rca
                #increase M_N
                #infoOut = "Increase M/N from " + str(M_N) + " to "
                newM_N = min (1, M_N * (1 + M_N_Change_Rate))
                M_N = newM_N
                elements.update_elements(newM_N)
                
                #output
                #infoOut = infoOut + str(newM_N)
                #print(infoOut)
                #elements.infoOut()    
                timePoint = []
                timePoint.append(timesThresholdFaults)
                timePoint.append(C_R/faultsTotal)
                timePoint.append(M_N)
                timePoint.append(delayTotal)
                M_N_history.append(timePoint)
                
            else:
                #output
                timePoint = []
                timePoint.append(timesThresholdFaults)
                timePoint.append(C_R/faultsTotal)
                timePoint.append(M_N)
                timePoint.append(delayTotal)
                M_N_history.append(timePoint)
            
            timesThresholdFaults = timesThresholdFaults + 1
 """       
 #Performance calculation    
 print()
 print("option = " + str(option))
 print("Total num of faults: " + str(faultsTotal) + ", total num of impacted faults = " + str(impactedFaultsTotal))
 print("C_M = " + str(C_M) + ", the ratio of monitored, but not detected faults = " + str(C_M/faultsTotal))
 #print("C_D = " + str(C_D) + ", the ratio of detected faults, but not diagnosed = " + str(C_D/faultsTotal))
 print("C_R = " + str(C_R) + ", the ratio of diagnosed faults = " + str(C_R/faultsTotal))
 print("FailureRate = " + str(failureRatio) + ", FaultRecoverDelay = " + str(delayTotal/faultsTotal))
 print()
 print("THRESHOLD_DOWN = " + str(THRESHOLD_DOWN) + " " + "THRESHOLD_UP = " + str(THRESHOLD_UP))
 print("M_N changing curve: # of diagnosedFaults, diagnosedFaults/totalFaults, M_N, totalDelay")
 for point in M_N_history:
    print (str(point[0]) + ", " + str(point[1]) + ", " + str(point[2]) + ", " + str(point[3]))
 """

 #output
 if option == 1 or option == 2:
        dataPoint = []
        dataPoint.append(M_N)
        dataPoint.append(accuracyRCA)
        dataPoint.append(delayTotal/faultsTotal)
        return dataPoint

 if option == 3 or option == 4:
        return M_N_history



In [117]:
LAMDAS = [0.01, 0.08, 0.3] #the fault generation rates for each experiment for each layer
M_NS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] #ratio of M/N
accuracyRCAs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] 
accuracyCorrelations = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] 


In [79]:
option = 1 #options

dataPoints = []
for M_N in M_NS:
    dataPoint = One_Experiment(option, LAMDAS[0], M_N, 0, 0)
    dataPoints.append(dataPoint)
    
infoOut = ""
for dataPoint in dataPoints:
    infoOut = infoOut + str(dataPoint[0]) + ", "
infoOut = infoOut + "\n"
for dataPoint in dataPoints:
    infoOut = infoOut + str(dataPoint[2]) + ", "
print(infoOut)

0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 
9.805769451787151, 9.604558142211735, 9.4013159785076, 9.200643530222937, 8.99422629477494, 8.793342340529508, 8.597650303047713, 8.395909926470589, 8.201023517911564, 8.0, 


In [102]:
option = 2

dataCurves = [] # multiple points for a curve
for M_N in M_NS:
    dataCurve = []
    for accuracyRCA in accuracyRCAs:
        dataPoint = One_Experiment(option, LAMDAS[0], M_N, accuracyRCA, 0)
        dataCurve.append(dataPoint)
    dataCurves.append(dataCurve)
  
infoOut = ""
for dataCurve in dataCurves:
    infoOut = infoOut + "M_N = " + str(dataCurve[0][0]) + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[1]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[2]) + ", "
    infoOut = infoOut + "\n"
    
print(infoOut)




M_N = 0.1
0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 
9.769166952740589, 9.737841858336195, 9.708319029637257, 9.68051264446733, 9.652963725826753, 9.619836365716901, 9.589798603959263, 9.559331731319373, 9.528864858679484, 9.49779723080444, 
M_N = 0.2
0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 
9.536734048365298, 9.471863588003565, 9.40914972539322, 9.35031773873537, 9.290968168617189, 9.230152112028065, 9.175374529142823, 9.114127153003421, 9.055812749805906, 8.998188457888835, 
M_N = 0.3
0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 
9.303522571819425, 9.21733926128591, 9.125256497948017, 9.035140218878249, 8.94656292749658, 8.856532147742818, 8.769750341997264, 8.682113543091655, 8.588064295485637, 8.496922024623803, 
M_N = 0.4
0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 
9.072839718261466, 8.946229170245662, 8.823569833361965, 8.70640783370555, 8.584521559869438, 8.46143274351486, 8.34847964267308, 8.226851056519498, 8.11157876653496, 7.986600240508504, 
M_N = 0.5


In [114]:
option = 3
M_N = 0.5
accuracyRCA = 0.9
dataCurves = []

dataCurve = One_Experiment(option, LAMDAS[0], M_N, accuracyRCA, 0)
dataCurves.append(dataCurve)
    
infoOut = ""
for dataCurve in dataCurves:
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[0]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[1]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[2]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[3]) + ", "
    infoOut = infoOut + "\n"
    
print(infoOut)




0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
0, 0.43600233781414377, 0.4410884741416156, 0.44691500625661756, 0.4451858229301191, 0.4470803965388803, 0.4468970080824313, 0.447943864229765, 0.4482117310443491, 0.44970451801486944, 
0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 
0, 26350, 53862, 79535, 106833, 133634, 162085, 187706, 214050, 240780, 



In [118]:
option = 4
M_N = 0.5
accuracyRCA = 0.9



dataCurves = [] 
for accuracyCorrelation in accuracyCorrelations:
    dataCurve = One_Experiment(option, LAMDAS[0], M_N, accuracyRCA, accuracyCorrelation)
    dataCurves.append(dataCurve)
  
infoOut = ""
for dataCurve in dataCurves:
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[0]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[1]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[2]) + ", "
    infoOut = infoOut + "\n"
    for dataPoint in dataCurve:
        infoOut = infoOut + str(dataPoint[3]) + ", "
    infoOut = infoOut + "\n"
    
print(infoOut)


0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
0, 0.46421609579504314, 0.5588445588445589, 0.6439800613496932, 0.689253498117764, 0.715383732935643, 0.7349495529096735, 0.7465388711395101, 0.7561376550048243, 0.763241843277218, 
0.5, 0.65, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 
0, 27191, 49458, 69188, 90152, 109264, 129042, 149170, 169611, 190749, 
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
0, 0.5275345947472465, 0.589440728618187, 0.6712949844148484, 0.7138828174575196, 0.7358211502360772, 0.751019053938762, 0.7643706039291778, 0.7726584240374793, 0.779179317036527, 
0.5, 0.65, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 0.8450000000000001, 
0, 25734, 48780, 69005, 89313, 108669, 128755, 149231, 170041, 190254, 
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
0, 0.5509013785790032, 0.6262359002924384, 0.64644469525